This corpus contains a large metadata-rich collection of fictional conversations extracted from raw movie scripts:



- 220,579 conversational exchanges between 10,292 pairs of movie characters

- involves 9,035 characters from 617 movies

- in total 304,713 utterances

- movie metadata included:

    - genres

    - release year

    - IMDB rating

    - number of IMDB votes

    - IMDB rating

- character metadata included:

    - gender (for 3,774 characters)

    - position on movie credits (3,321 characters)

- see README.txt (included) for details

https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

1.가상환경 설치

conda  create -n venv_chatbot python=3.5 anaconda

2.가상환경 실행
conda activate venv_chatbot

3.가상환경 종료
conda deactivate



Tensorfow 1.0.0 설치
Pip install tensorflow==1.0.0


In [1]:
import numpy as np
import tensorflow as tf
import re
import time

In [3]:
pwd

'/Users/kimkwangil/Documents/Projects/chatbot'

In [5]:
ls

chatbot_.ipynb                    movie_conversations.txt
cornell movie-dialogs corpus/     movie_lines.txt
cornell_movie_dialogs_corpus.zip


In [6]:
lines = open('movie_lines.txt', encoding='utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding='utf-8', errors = 'ignore').read().split('\n')

In [7]:
lines[:5] #대화 데이터 확인

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."]

In [8]:
conversations[:5] #대화구성 확인

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']"]

In [15]:
conversations[:-1] #맨 뒷부분 아이템 확인

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L367', 'L368']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L401', 'L402', 'L403']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L404', 'L405', 'L406', 'L407']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L575', 'L576']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L577', 'L578']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L662', 'L663']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L693', 'L69

In [11]:
#creating a dictionary. each line and id
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ') #한 줄을 불러와서 +++$+++ 로 분리
    if len(_line) == 5:
        id2line[_line[0]] = _line[4] #'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!' 4번째가 대화로 이것을 딕셔너리에 저장
        

In [14]:
id2line # 전처리 완료  key, value로 구성

{'L393477': 'I want to make that come to life-- yes, and lighted up like that, too-- for every boy in the land. Boys forget what their country means--just reading "land of the free" in history books. And they get to be men--and forget even more. Liberty is too precious to get buried in books, Miss Saunders. Men ought to hold it up in front of them--every day of their lives and say: "I am free--to think--to speak. My ancestors couldn\'t. I can. My children will."',
 'L10772': "In this case, getting married means nothing. Aren't we already the same as being married? And Corrado and Giulia -- aren't they already the same as being married?",
 'L626678': '...we had to lick his boots clean.',
 'L157239': "What's his specialty, brown-nosing?",
 'L543155': 'Las Vegas, please. I do not speak English.',
 'L184545': "Yes, I've been asking for weeks, a month, I don't know, also my leg was -",
 'L587971': 'Of course, there was a lot more room before they stuck the memorial down here.',
 'L642821': 

In [16]:
# 대화 구성 전처리
conversations_ids = []
for conversation in conversations[:-1]:
    # "u5 +++$+++ u8 +++$+++ m0 +++$+++ ['L103', 'L104', 'L105', 'L106', 'L107']" 에서 대화내용만 추출
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ", "")
    conversations_ids.append(_conversation.split(',')) # , 로 구분해놓는다.

In [17]:
conversations_ids[:5] #전처리 확인

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208']]

In [18]:
#이제 질문과 답변을 구분!
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]]) # 대화구성 내용을 한줄씩 가져와서 각 대화를 id2line에서 찾아서 첫번째것이 질문이고
        answers.append(id2line[conversation[i+1]]) # 두 번째 것이 답변. 이것을 각 questions, answers에 넣음

In [21]:
questions[:1] #check for questions 잘됨

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.']

In [22]:
answers[:1]#check for answers 잘됨

["Well, I thought we'd start with pronunciation, if that's okay with you."]

In [24]:
# 이러한 questions, answers는 학습시키는데 에러가 발생함, 그래서 조금 더 다음어야함 cleaning!
def clean_text(text):
    text = text.lower()# 소문자로 전환
    text = re.sub(r"i'm", "i am", text) #약어는 풀어서 표현
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"/'ll", "will", text)
    text = re.sub(r"/'ve", "have", text)
    text = re.sub(r"/'re", "are", text)
    text = re.sub(r"/'d", "would", text)
    text = re.sub(r"/'won't", "will not", text)
    text = re.sub(r"/'can't", "can not", text)
    text = re.sub(r"/[-()\"#/;:<>{}+=-~|.?,]", "", text)
    return text

In [25]:
#cleaning the questions
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question)) # 질문을 가져와서 함수로 클리닝 하고 리스트로 저장함
    
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [26]:
clean_questions[:5] # 크리닝 잘됨

['can we make this quick?  roxanne korrine and andrew barrett are having an incredibly horrendous public break- up on the quad.  again.',
 "well, i thought we'd start with pronunciation, if that is okay with you.",
 'not the hacking and gagging and spitting part.  please.',
 "you're asking me out.  that is so cute. what's your name again?",
 "no, no, it's my fault -- we didn't have a proper introduction ---"]

In [27]:
clean_answers[:5] # 크리닝 잘됨

["well, i thought we'd start with pronunciation, if that is okay with you.",
 'not the hacking and gagging and spitting part.  please.',
 "okay... then how 'bout we try out some french cuisine.  saturday?  night?",
 'forget it.',
 'cameron.']

In [28]:
# Create Dictionary 각 단어들이 얼마나 등장하는지 사전에 담기
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [29]:
word2count #questions, answers에 포함된 모든 단어의 빈도수 계산완료!

{'connive': 1,
 'nutriment!': 1,
 'flyers.': 1,
 'product!': 2,
 'luc.': 11,
 'fit.': 53,
 'generated': 11,
 'god...you': 3,
 'mook': 2,
 'rehearsal?': 2,
 'friedman,': 6,
 'vindicated!': 2,
 'envelopes': 2,
 'believing': 36,
 'hmmm...44': 1,
 'tank...': 2,
 'wakes.': 2,
 'doctorates,': 2,
 'semester,': 3,
 'read?': 27,
 '--sacred': 1,
 'loomis,': 5,
 'missed!': 4,
 'hydrochloric': 1,
 'esau....': 1,
 'creed': 21,
 'cal!': 2,
 'milt,': 2,
 'warned.': 8,
 'captain.....': 1,
 "rory's": 1,
 'seventies.': 2,
 'answers?': 14,
 'vampires...': 2,
 'doin\'?"': 1,
 'societies.': 4,
 'row!!': 1,
 "margaret's": 2,
 'faithfully': 3,
 'guilt': 26,
 'elbow.': 11,
 'destiny,': 18,
 'wo-wo': 1,
 'scared!!!': 1,
 'weighs?': 1,
 "santos'": 2,
 'unprocessed': 2,
 'brutal.': 4,
 'papillomavirus.': 1,
 'buzzard': 3,
 '-we': 2,
 "gorbachev's": 1,
 '1879.': 1,
 'letter...reckon': 1,
 'juan.': 2,
 'pastry,': 2,
 'conversations?': 2,
 'vial?': 2,
 "weren't...": 3,
 'squirming': 1,
 'munched...': 1,
 'domestica

In [36]:
word2count.items() #딕셔너리의 아이템을 확인한다.

dict_items([('connive', 1), ('nutriment!', 1), ('flyers.', 1), ('product!', 2), ('luc.', 11), ('fit.', 53), ('generated', 11), ('god...you', 3), ('mook', 2), ('rehearsal?', 2), ('friedman,', 6), ('vindicated!', 2), ('envelopes', 2), ('believing', 36), ('hmmm...44', 1), ('tank...', 2), ('wakes.', 2), ('doctorates,', 2), ('semester,', 3), ('read?', 27), ('--sacred', 1), ('loomis,', 5), ('missed!', 4), ('hydrochloric', 1), ('esau....', 1), ('creed', 21), ('cal!', 2), ('milt,', 2), ('warned.', 8), ('captain.....', 1), ("rory's", 1), ('seventies.', 2), ('answers?', 14), ('vampires...', 2), ('doin\'?"', 1), ('societies.', 4), ('row!!', 1), ("margaret's", 2), ('faithfully', 3), ('guilt', 26), ('elbow.', 11), ('destiny,', 18), ('wo-wo', 1), ('scared!!!', 1), ('weighs?', 1), ("santos'", 2), ('unprocessed', 2), ('brutal.', 4), ('papillomavirus.', 1), ('buzzard', 3), ('-we', 2), ("gorbachev's", 1), ('1879.', 1), ('letter...reckon', 1), ('juan.', 2), ('pastry,', 2), ('conversations?', 2), ('vial?'

In [32]:
#두개의 딕셔너리에 질문, 답변의 단어를 유일한 숫자로 변환하여 제작
threshold = 20 # 보통 20번 이상 나오는 단어는 중요한 단어니까. 이것을 기준으로 각 단어들을 유일한 넘버를 매겨서 딕셔너리로 만들자. 20은 임의 기준값

questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        questionswords2int[word] = word_number
        word_number += 1
        
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold:
        answerswords2int[word] = word_number
        word_number += 1

In [34]:
questionswords2int #단어를 유일한 숫자로 변환 성공

{'captain.': 10382,
 'compete': 7161,
 'fit.': 0,
 'speak,': 7162,
 'guilty': 4288,
 "runnin'": 7163,
 'picture': 4289,
 'other...': 5626,
 'intimate': 9966,
 'junior': 1501,
 'believing': 1,
 'dinner.': 4290,
 'simple.': 11142,
 'england.': 7164,
 'read?': 2,
 'forever.': 5627,
 'day': 7822,
 'creed': 3,
 'heart...': 7,
 'necessary': 9967,
 'spring': 7166,
 'learning': 8520,
 'jail.': 9970,
 'it....': 4291,
 'testing': 10997,
 'eventually.': 9206,
 'guilt': 4,
 'apartment,': 3816,
 'because...': 1502,
 'clown': 8510,
 'bomb': 9971,
 'federation': 6173,
 'power.': 5628,
 'kent.': 5629,
 'expose': 4292,
 'instinct.': 9965,
 'shes': 8532,
 'tell': 2933,
 'he.': 8512,
 'yes...': 1504,
 'camp': 4293,
 'presume': 4294,
 'bond': 10502,
 '"do': 8514,
 'part.': 2923,
 'private.': 1505,
 'italy': 1506,
 'usual': 2924,
 'check.': 2940,
 'can.': 1507,
 'discuss': 1508,
 'from...': 613,
 'around,': 5,
 'attacked.': 5904,
 'justice': 2925,
 'like.': 7168,
 'vacation,': 1509,
 'hollywood': 5634,
 'p

In [35]:
answerswords2int

{'captain.': 10382,
 'compete': 7161,
 'fit.': 0,
 'speak,': 7162,
 'guilty': 4288,
 "runnin'": 7163,
 'picture': 4289,
 'other...': 5626,
 'intimate': 9966,
 'junior': 1501,
 'believing': 1,
 'dinner.': 4290,
 'simple.': 11142,
 'england.': 7164,
 'read?': 2,
 'forever.': 5627,
 'day': 7822,
 'creed': 3,
 'heart...': 7,
 'necessary': 9967,
 'spring': 7166,
 'learning': 8520,
 'jail.': 9970,
 'it....': 4291,
 'testing': 10997,
 'eventually.': 9206,
 'guilt': 4,
 'apartment,': 3816,
 'because...': 1502,
 'clown': 8510,
 'bomb': 9971,
 'federation': 6173,
 'power.': 5628,
 'kent.': 5629,
 'expose': 4292,
 'instinct.': 9965,
 'shes': 8532,
 'tell': 2933,
 'he.': 8512,
 'yes...': 1504,
 'camp': 4293,
 'presume': 4294,
 'bond': 10502,
 '"do': 8514,
 'part.': 2923,
 'private.': 1505,
 'italy': 1506,
 'usual': 2924,
 'check.': 2940,
 'can.': 1507,
 'discuss': 1508,
 'from...': 613,
 'around,': 5,
 'attacked.': 5904,
 'justice': 2925,
 'like.': 7168,
 'vacation,': 1509,
 'hollywood': 5634,
 'p

In [40]:
len(questionswords2int)

11449

In [38]:
#질문, 답변 Dictionary에 중요한 토큰 4개를 입력(적용)함. 키와 값은 다음과 같음...

# EOS - "end of string 
# UNK - "unknown token" 
# PAD - your GPU (or CPU at worst) processes your training data in batches
# SOS - "Start of String"
# OUT - "filter out"

tokens = ['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1

In [39]:
questionswords2int

{'captain.': 10382,
 'compete': 7161,
 'fit.': 0,
 'speak,': 7162,
 'guilty': 4288,
 "runnin'": 7163,
 'picture': 4289,
 'other...': 5626,
 'intimate': 9966,
 'junior': 1501,
 'believing': 1,
 'dinner.': 4290,
 'simple.': 11142,
 'england.': 7164,
 'read?': 2,
 'forever.': 5627,
 'day': 7822,
 'creed': 3,
 'heart...': 7,
 'necessary': 9967,
 'spring': 7166,
 'learning': 8520,
 'jail.': 9970,
 'it....': 4291,
 'testing': 10997,
 'eventually.': 9206,
 'guilt': 4,
 'apartment,': 3816,
 'because...': 1502,
 'clown': 8510,
 'bomb': 9971,
 'federation': 6173,
 'power.': 5628,
 'kent.': 5629,
 'expose': 4292,
 'instinct.': 9965,
 'shes': 8532,
 'tell': 2933,
 'he.': 8512,
 'yes...': 1504,
 'camp': 4293,
 'presume': 4294,
 'bond': 10502,
 '"do': 8514,
 'part.': 2923,
 'private.': 1505,
 'italy': 1506,
 'usual': 2924,
 'check.': 2940,
 'can.': 1507,
 'discuss': 1508,
 'from...': 613,
 'around,': 5,
 'attacked.': 5904,
 'justice': 2925,
 'like.': 7168,
 'vacation,': 1509,
 'hollywood': 5634,
 'p

In [42]:
#making inverse dictionary of the answers2int dic  이렇게해야 숫자로 변환한 단어를 처리가능
answersints2word = {w_i: w for w, w_i in answerswords2int.items()} #딕셔너리 순서바꿈
answersints2word

{0: 'fit.',
 1: 'believing',
 2: 'read?',
 3: 'creed',
 4: 'guilt',
 5: 'around,',
 6: 'paper?',
 7: 'heart...',
 8: 'this?!',
 9: 'have...',
 10: 'killing',
 11: 'ready,',
 12: 'skipper.',
 13: 'possibilities',
 14: 'bond.',
 15: 'caught',
 16: 'dangerous',
 17: 'market,',
 18: 'house.',
 19: 'mommy.',
 20: 'sleeping.',
 21: 'expand',
 22: 'transferred',
 23: 'chicken?',
 24: 'say',
 25: 'darn',
 26: 'clock.',
 27: 'sucker',
 28: 'delay',
 29: 'effect',
 30: 'guy!',
 31: 'regarding',
 32: 'naturally',
 33: 'french',
 34: "needn't",
 35: 'lady?',
 36: 'cleaned',
 37: 'remember.',
 38: 'au',
 39: 'princess.',
 40: 'there?',
 41: 'start',
 42: 'cops.',
 43: 'mind',
 44: 'naive',
 45: 'hopefully',
 46: 'gay?',
 47: 'choices.',
 48: 'weather,',
 49: 'kirk',
 50: 'cigarette',
 51: 'artist',
 52: 'pressure',
 53: 'detailed',
 54: 'distance',
 55: 'liar!',
 56: 'solve',
 57: 'heads',
 58: '...so',
 59: 'engineering',
 60: 'signs',
 61: 'laid.',
 62: 'witness.',
 63: 'tail',
 64: 'jail',
 65: 

In [43]:
# EOS 토큰을 모든 답변 뒤에 붙이기. 
for i in range(len(clean_answers)):
    clean_answers[i] += ' <EOS>' 

In [44]:
clean_answers[:5] #모든 답변 문장 뒤에 문장이 끝났다는 토큰 적용 완료!

["well, i thought we'd start with pronunciation, if that is okay with you. <EOS>",
 'not the hacking and gagging and spitting part.  please. <EOS>',
 "okay... then how 'bout we try out some french cuisine.  saturday?  night? <EOS>",
 'forget it. <EOS>',
 'cameron. <EOS>']

In [45]:
#이제 모든 문장을 숫자로 변환하자
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)
    
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in question.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)

In [48]:
questions_into_int[:3] #모든 문장이 숫자로 변경됨

[[7731,
  11080,
  9521,
  8200,
  11448,
  11448,
  11448,
  11044,
  11077,
  11448,
  130,
  8245,
  5542,
  4795,
  11448,
  2375,
  11448,
  10983,
  1022,
  320,
  11448,
  674],
 [9973,
  4870,
  3506,
  1146,
  41,
  11090,
  11448,
  1851,
  11119,
  460,
  4213,
  11090,
  2234],
 [6559, 320, 11448, 11044, 11448, 11044, 11448, 2923, 6231]]

In [49]:
answers_into_int[:2] #답변도 모두 숫자로 변경됨

[[11044,
  4870,
  8960,
  2536,
  6242,
  5219,
  6559,
  3965,
  3598,
  5025,
  2046,
  10445,
  9830,
  4569,
  6571,
  10317,
  3158,
  1955,
  4569,
  11448,
  11448],
 [11044,
  4870,
  8960,
  2536,
  6242,
  5219,
  6559,
  3965,
  3598,
  5025,
  2046,
  10445,
  9830,
  4569,
  6571,
  10317,
  3158,
  1955,
  4569,
  11448,
  11448]]

In [50]:
#질문의 길이별로 질문, 답변 재구성하기
sorted_clean_questions = []
sorted_clean_answers =[]
for length in range(1, 25 + 1): #25개의 단어 길이로 반복
    for i in enumerate(questions_into_int): # 숫자로 바뀐 문장의 숫자(단어)를 하나씩 꺼내서
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [52]:
sorted_clean_questions[:4] # 단어의 길이대로 정렬됨.. 확인해보면 알게됨 ㅎㅎㅎ 지금은 4개만 확인

[[11448], [3698], [3871], [5507]]

In [53]:
sorted_clean_answers[:5]

[[11044,
  4870,
  8960,
  2536,
  6242,
  5219,
  6559,
  3965,
  3598,
  5025,
  2046,
  10445,
  9830,
  4569,
  6571,
  10317,
  3158,
  1955,
  4569,
  11448,
  11448],
 [11044,
  4870,
  8960,
  2536,
  6242,
  5219,
  6559,
  3965,
  3598,
  5025,
  2046,
  10445,
  9830,
  4569,
  6571,
  10317,
  3158,
  1955,
  4569,
  11448,
  11448],
 [11044,
  4870,
  8960,
  2536,
  6242,
  5219,
  6559,
  3965,
  3598,
  5025,
  2046,
  10445,
  9830,
  4569,
  6571,
  10317,
  3158,
  1955,
  4569,
  11448,
  11448],
 [11044,
  4870,
  8960,
  2536,
  6242,
  5219,
  6559,
  3965,
  3598,
  5025,
  2046,
  10445,
  9830,
  4569,
  6571,
  10317,
  3158,
  1955,
  4569,
  11448,
  11448],
 [11044,
  4870,
  8960,
  2536,
  6242,
  5219,
  6559,
  3965,
  3598,
  5025,
  2046,
  10445,
  9830,
  4569,
  6571,
  10317,
  3158,
  1955,
  4569,
  11448,
  11448]]